In [ ]:
from server import *
from prompt_template.prompt_rephrase import *
from global_functions import *

from games.guessing_game import *
from games.bar_game import *
from games.pirate_game import *
from games.diner_dilemma import *
from games.divide_dollar import *
from games.sealed_bid_auction import *
from games.public_goods import *
from games.battle_royale import *

In [ ]:
# Rephrase demo
rephrase_files = ['description', 'report_explicit', 'report_implicit', 'request']
rephrase([f"prompt_template/bar_game/{f}_v1.txt" for f in rephrase_files])

In [ ]:
# Game Configuration
player_num = 10
models = ['gpt-3.5-turbo' if i%2==0 else 'gpt-4' for i in range(player_num)]
models = ['gpt-3.5-turbo' for i in range(player_num)]
version = 'v1'

In [ ]:
# Guessing Game
game = GuessingGame(player_num=10, min=0, max=100, ratio=2/3, ratio_str='2/3', version='v1')
game.run(20)

# Load a game
# game = load('save/guessing_game.json', GuessingGame, 'guessing_game2')  # load and rename the saved data 
# game.run(20)

# # Plot the graph
# game.show('model', ['gpt-3.5-turbo'])
# game.show('id', [f"player_{i}" for i in range(player_num) if i%2==0])

In [ ]:
# Bar Game
## Explicit Game
bargame_explicit = BarGame(player_num=10, min=0, max=10, home=5, ratio=0.6, ratio_str='60%',
                           version='v1', mode='explicit', name_exp='bargame_explicit')
bargame_explicit.run(20)

## Implicit Game
bargame_implicit = BarGame(player_num=10, min=0, max=10, home=5, ratio=0.6, ratio_str='60%',
                           version='v1', mode='explicit', name_exp='bargame_implicit')
bargame_implicit.run(20)

In [ ]:
# Pirate Game
pirate_game = PirateGame(player_num=10, gold=100, version='v1')
pirate_game.run(10)

In [ ]:
# Divide the Dollar Game
divide_dollar = DivideDollar(player_num=10, golds=100, version='v1')
divide_dollar.run(20)

In [ ]:
# Diner Dilemma
cheap_cost, cheap_utility, exp_cost, exp_utility = 20, 40, 50, 60
cheap_cost, cheap_utility, exp_cost, exp_utility = 5, 15, 10, 17

diner_dilemma = DinerDilemma(player_num=20, cheap_cost=20, cheap_utility=40,
                             exp_cost=50, exp_utility=60, version='v1')
diner_dilemma.run(20)

In [ ]:
# Sealed Bid auction 
sealed_bid_auction = SealedBidAuction(player_num=10, mode="second-highest bid", valuation=100, version='v1')
sealed_bid_auction.run(20)

In [ ]:
# Public Goods Game
public_goods_game = PublicGoods(player_num=10, tokens=100, ratio=random.randint(1, 10 / 2), version='v1')
public_goods_game.run(20)

In [ ]:
# Battle Royale
battle_royale = BattleRoyale(player_num=10, version='v1')
battle_royale.run(20)